<a href="https://colab.research.google.com/github/scorepia/Realtime6005_Project/blob/main/DADS6005_Project_producer1_PEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!pip install yfinance --upgrade --no-cache-dir

!pip install confluent_kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing yahoo finance API

In [79]:
import yfinance as yf

Dumping PEP stock data 

In [80]:
pep = yf.download("PEP", period = "1y" , interval = "1d")

[*********************100%***********************]  1 of 1 completed


In [40]:
print(type(pep))

pep.info()
pep.head()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-01-13 to 2023-01-12
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       251 non-null    float64
 1   High       251 non-null    float64
 2   Low        251 non-null    float64
 3   Close      251 non-null    float64
 4   Adj Close  251 non-null    float64
 5   Volume     251 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.7 KB


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-13,173.710007,174.690002,173.240005,174.179993,169.641266,4310300
2022-01-14,174.570007,175.940002,173.399994,175.639999,171.063232,4474500
2022-01-18,173.619995,174.759995,173.149994,173.960007,169.426987,5789500
2022-01-19,173.960007,175.960007,173.570007,175.210007,170.644440,6424500
2022-01-20,175.410004,176.880005,173.839996,173.940002,169.407516,6284200


In [41]:
pep1 = pep.reset_index()
pep1 = pep1.rename(columns = {'Close':'Close_Price'})
pep1 = pep1[['Date','Close_Price','Volume']]

pep1.info()
pep1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         251 non-null    datetime64[ns]
 1   Close_Price  251 non-null    float64       
 2   Volume       251 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 6.0 KB


,Date,Close_Price,Volume
0,2022-01-13,174.179993,4310300
1,2022-01-14,175.639999,4474500
2,2022-01-18,173.960007,5789500
3,2022-01-19,175.210007,6424500
4,2022-01-20,173.940002,6284200


Preparing data for Producer

In [84]:
Lines = []
n = 250

for i in range(n+1):
    j = pep1.loc[i]
    js_data = j.to_json()
    bt_data = js_data.encode()
    Lines.append(js_data)

Sending to AWS

In [85]:
from confluent_kafka import Producer
import time
import json
from uuid import uuid4

p = Producer({'bootstrap.servers': 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'})

def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {}'.format(msg.value().decode('utf-8')))

for line in Lines:
    p.poll(0)
    sendMsg = line #line.encode().decode('utf-8').strip('\n')
    p.produce('producer1', key=str(uuid4()), value=sendMsg , callback=delivery_report)
    '''time.sleep(0.2)'''

p.flush()

Message delivered to {"Date":1642032000000,"Close_Price":174.1799926758,"Volume":4310300}
Message delivered to {"Date":1642118400000,"Close_Price":175.6399993896,"Volume":4474500}
Message delivered to {"Date":1642464000000,"Close_Price":173.9600067139,"Volume":5789500}
Message delivered to {"Date":1642550400000,"Close_Price":175.2100067139,"Volume":6424500}
Message delivered to {"Date":1642636800000,"Close_Price":173.9400024414,"Volume":6284200}
Message delivered to {"Date":1642723200000,"Close_Price":174.2200012207,"Volume":6753200}
Message delivered to {"Date":1642982400000,"Close_Price":173.3300018311,"Volume":9263900}
Message delivered to {"Date":1643068800000,"Close_Price":171.3399963379,"Volume":6536900}
Message delivered to {"Date":1643155200000,"Close_Price":169.5299987793,"Volume":6841800}
Message delivered to {"Date":1643241600000,"Close_Price":169.3699951172,"Volume":6543500}
Message delivered to {"Date":1643328000000,"Close_Price":172.6699981689,"Volume":5796000}
Message de

0